# Import Libraries

In [ ]:
import os, random, shutil, torch, cv2
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from torchvision.transforms.functional import to_tensor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision.models import vgg16
from torchvision import transforms as T
import torch.nn.functional as F

# Setting Up Directories and Import Dataset

In [ ]:
source_dir = r"C:\Users\atole\OneDrive\Desktop\Python\archive(1)\Image Super Resolution - Unsplash\high res"
target_root = r"\Users\atole\OneDrive\Desktop\Python\Working Dataset"
os.makedirs(target_root, exist_ok=True)

# Setting Up Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

# Importing Device

In [ ]:
import requests
url = "https://raw.githubusercontent.com/swz30/Restormer/main/basicsr/models/archs/restormer_arch.py"
save_path = "restormer_arch.py"
response = requests.get(url)
if response.status_code == 200:
    with open(save_path, "wb") as f:
        f.write(response.content)
    print(f"Saved to {save_path}")
else:
    print(f" Failed to download. Status code: {response.status_code}")

# Loading Restormer with its weights

In [ ]:
import sys
!pip install einops
sys.path.append(r'C:\Users\atole\OneDrive\Desktop\Python\Working Dataset')
from restormer_arch import Restormer
import torch

teacher = Restormer(
    inp_channels=3,               
    out_channels=3,
    dim=48,
    num_blocks=[4, 6, 6, 8],
    num_refinement_blocks=4,
    heads=[1, 2, 4, 8],
    ffn_expansion_factor=2.66,
    bias=False,
    LayerNorm_type='WithBias',
    dual_pixel_task=False        
)


weights = torch.load(r"C:\Users\atole\OneDrive\Desktop\Python\single_image_defocus_deblurring.pth", map_location=device)
teacher.load_state_dict(weights.get("params", weights))
teacher = teacher.to(device).eval()

print(" Restormer (3-channel teacher for single-image defocus deblurring) ready")

# Defining the Student Model

In [ ]:
class StudentCNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3, padding=1), torch.nn.ReLU(),
            torch.nn.Conv2d(32, 64, 3, padding=1), torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(64, 64, 3, padding=1), torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 3, padding=1), torch.nn.ReLU(),
            torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Conv2d(64, 32, 3, padding=1), torch.nn.ReLU(),
            torch.nn.Conv2d(32, 3, 3, padding=1), torch.nn.Tanh()  
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return (x + 1) / 2

student = StudentCNN().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)
criterion = torch.nn.L1Loss()

# Perceptual Loss Definition

In [ ]:
vgg = vgg16(pretrained=True).features.to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

vgg_normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])

def perceptual_loss(pred, target):
    pred_resized = F.interpolate(pred, size=(224, 224), mode='bilinear', align_corners=False)
    target_resized = F.interpolate(target, size=(224, 224), mode='bilinear', align_corners=False)
    pred_norm = vgg_normalize(pred_resized.squeeze(0)).unsqueeze(0)
    target_norm = vgg_normalize(target_resized.squeeze(0)).unsqueeze(0)
    pred_feat = vgg(pred_norm)
    target_feat = vgg(target_norm)
    return F.l1_loss(pred_feat, target_feat)

# Training loop for Set B of images & Saving weights

In [ ]:
train_path = os.path.join(target_root, "train_B")  
train_files = os.listdir(train_path)
student.train()
teacher.eval()
best_ssim = 0.0

for epoch in range(15):
    total_loss = 0
    student.train()
    ssim_total = 0.0

    for fname in tqdm(train_files, desc=f" Training Student | Epoch {epoch+1}"):
        img = cv2.imread(os.path.join(train_path, fname))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        h, w = h - (h % 8), w - (w % 8)
        gt = cv2.resize(img, (w, h))
        lr = cv2.resize(gt, (w//2, h//2), interpolation=cv2.INTER_LINEAR)
        input_img = cv2.resize(lr, (w, h), interpolation=cv2.INTER_CUBIC)

        input_tensor = to_tensor(input_img).unsqueeze(0).to(device)
        gt_tensor = to_tensor(gt).unsqueeze(0).to(device)

        with torch.no_grad():
            target_tensor = teacher(input_tensor)

        output = student(input_tensor)

        l1 = criterion(output, target_tensor)
        gt_l1 = criterion(output, gt_tensor)
        p_loss = perceptual_loss(output, gt_tensor)
        loss = 0.9 * l1 + 0.1 * gt_l1 + 0.005 * p_loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        with torch.no_grad():
            s_output = output.squeeze(0).clamp(0, 1).cpu().numpy()
            s_gt = to_tensor(gt).numpy()
            s_output = np.transpose(s_output, (1, 2, 0))
            s_gt = np.transpose(s_gt, (1, 2, 0))
            score = ssim(s_output, s_gt, channel_axis=2, data_range=1.0, win_size=11)
            ssim_total += score

    avg_loss = total_loss / len(train_files)
    avg_ssim = ssim_total / len(train_files)
    print(f" Epoch {epoch+1} | Avg L1 Loss: {avg_loss:.6f} | Avg SSIM: {avg_ssim:.4f}")

if avg_ssim > best_ssim:
        best_ssim = avg_ssim
        torch.save(student.state_dict(), "best_student.pth")
        print(f" Best model saved at Epoch {epoch+1} with SSIM: {best_ssim:.4f}")